In [4]:
def gen_paragraphs(settings, section_id, chapter_id):
    section = next((section for section in settings['sections'] if section['id'] == section_id), None);
    chapter = next((chapter for chapter in section['chapters'] if chapter['id'] == chapter_id), None);
    response = openai.chat.completions.create(
        model= settings["model"],
        messages= [
            {
                "role": "system",
                "content": 
                    f"""
                    You are an expert historian that specializes in building book projects.
                    Your task is to explode a chapter into a list of paragraphs to comprehensively exaust such chapter.
                    The tone of voice for the book is: {settings['tone_of_voice']}.
                    The length for a paragraph is between {settings['paragraph_length_min']} and {settings['paragraph_length_max']} words.
                    """
            },
            {
                "role": "system",
                "content": 
                    """
                    The output is a JSON document like:

                    {
                      "paragraphs": [
                        {
                          "id": 1,
                          "content": "content of the paragraph..."
                        },
                        {
                          "id": 2,
                          "content": "content of the paragraph..."
                        }
                      ]
                    }

                    The ID of the paragraph is a progressive number within this chapter.
                    """
            },
            {
                "role": "user",
                "content":
                    f"""
                    This is the general information about the book i'm writing:

                    Subject: {settings['subject']}
                    Title: {settings['title']}
                    Subtitle: {settings['subtitle']}

                    This is the full outline of the book so far:
                    {settings['sections']}

                    The section of the book that we are working on is:
                    {section}

                    The chapter of the book that you should develop is:
                    {chapter}

                    Generate between {settings['paragraphs_min']} and {settings['paragraphs_max']} paragraphs for the provided chapter.
                    """
            }
        ]
    )
    data = text2json(response.choices[0].message.content)
    return data